# 利用分类标签比对的方法间接实现

## 用预训练模型辅助标注（先使用这个）
实现方法：
使用目标检测模型（如 YOLO、Faster R-CNN）对图片进行初步检测和分类。
自动生成的标签再由人工快速校验和修改。\
工具：
使用 百度飞桨（PaddleDetection） 或 PyTorch Hub 提供的目标检测模型。
使用 TensorFlow Object Detection API 提供的预训练模型。

实现思路
使用 YOLO 检测图像中的物体

YOLO 的目标是检测图像中的目标，并生成：
物体类别标签：如 "car"（汽车）、"dog"（狗）。
检测框（Bounding Boxes）：标出目标的位置。
置信度：预测结果的可信度。
为每张图像生成一个标签集合

将 YOLO 检测出的所有目标类别汇总，形成一个标签集合，例如：
图像 A：["car", "person", "dog"]
图像 B：["cat", "tree", "person"]
构建一个图像数据库

对于需要检索的图像库，使用 YOLO 检测每张图像并存储结果：
图像路径。
物体标签集合。
（可选）目标的具体位置信息（检测框）。
检索流程

输入一张待检索的查询图像：
用 YOLO 检测图像，生成该图像的标签集合（如 ["car", "tree"]）。
与数据库中的标签集合进行比对，计算标签相似度（例如交集的大小）。
返回标签最相似的图像作为检索结果。
展示匹配结果

对检索出的图像，根据相似度排序，展示匹配图像及其检测到的目标（包括检测框）。

思路扩展
目标检测框的定义

每个检测目标由以下数据定义：
标签：目标类别名称（如 "car"、"person"）。
检测框：(x_min, y_min, x_max, y_max)，表示目标在图像中的位置。
置信度：目标预测的可信度。
位置信息的匹配方式


基于位置匹配可以采用如下算法：
检测框的交集面积
检测框的并集面积
IoU= 
检测框的并集面积
检测框的交集面积
​
 
距离衡量：计算检测框中心点之间的欧几里得距离。
权重综合：结合标签相似度和位置信息计算综合匹配得分。
匹配结果排序

使用目标标签匹配作为第一层过滤条件。
在候选图像中，计算所有匹配目标的位置信息相似度（如 IoU）。
综合匹配得分，按相似度排序。

In [1]:
from PIL import Image
import torch
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 加载 YOLO 模型
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to(device)

# 检测图像中的目标
def detect_objects(image_path):
    #print(image_path)
    results = model(image_path)
    objects = []
    for _, row in results.pandas().xyxy[0].iterrows():
        objects.append({
            "label": row['name'],  # 目标类别
            "bbox": [row['xmin'], row['ymin'], row['xmax'], row['ymax']],  # 检测框
            "confidence": row['confidence']  # 置信度
        })
    return objects

# 示例
query_image = "D:\picture\Training\query.jpg"
query_image_labels = detect_objects(query_image)
print(query_image_labels) 


Using cache found in C:\Users\10843/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-7 Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (Quadro T2000, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


[{'label': 'person', 'bbox': [849.5234985351562, 2141.595703125, 3496.396484375, 5736.5302734375], 'confidence': 0.8936266899108887}, {'label': 'car', 'bbox': [0.0, 3454.009521484375, 1238.6787109375, 5760.0], 'confidence': 0.47308674454689026}, {'label': 'person', 'bbox': [1.1253690719604492, 3883.7490234375, 417.5061950683594, 4509.42578125], 'confidence': 0.318872332572937}]


In [13]:
import json
import glob

# 构建数据库
def build_database(image_folder):
    database = {}
    image_paths = glob.glob(f"{image_folder}/*.jpg")
    for image_path in image_paths:
        objects = detect_objects(image_path)
        database[image_path] = objects
    return database

# 保存为 JSON 文件
database = build_database("D:\picture\Training")
with open("database.json", "w") as f:
    json.dump(database, f)


In [14]:
# 2. 计算 IoU
def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    # 计算交集面积
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    
    # 计算并集面积
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    
    # 避免除零错误
    if union_area == 0:
        return 0
    
    # 返回 IoU
    return inter_area / union_area

In [18]:
# 加载数据库
with open("database.json", "r") as f:
    database = json.load(f)

#综合匹配度计算
def match_images(query_objects, database):
    results = []
    
    for image_path, db_objects in database.items():
        total_similarity = 0
        for q_obj in query_objects:
            for db_obj in db_objects:
                if q_obj['label'] == db_obj['label']:  # 标签匹配
                    iou = calculate_iou(q_obj['bbox'], db_obj['bbox'])
                    total_similarity += iou  # 加入 IoU 相似度
        results.append((image_path, total_similarity))
    
    # 按相似度排序
    results = sorted(results, key=lambda x: x[1], reverse=True)
    return results

# 查询
query_objects = detect_objects("D:\picture\Training\query.jpg")
matches = match_images(query_objects, database)
#matches = search_image(query_labels, database)
print(matches)  # 返回匹配结果


D:\picture\Training\query.jpg
[('D:\\picture\\Training\\query.jpg', 3.0), ('D:\\picture\\Training\\IMG_6996.jpg', 0.7849665607487122), ('D:\\picture\\Training\\IMG_7018.jpg', 0.7726230583620426), ('D:\\picture\\Training\\IMG_7003.jpg', 0.7667829634435253), ('D:\\picture\\Training\\IMG_7015.jpg', 0.7517026128345199), ('D:\\picture\\Training\\IMG_7002.jpg', 0.6971159001375453), ('D:\\picture\\Training\\IMG_6984.jpg', 0.6633382202988782), ('D:\\picture\\Training\\IMG_7014.jpg', 0.6105143000587029), ('D:\\picture\\Training\\IMG_7012.jpg', 0.5381953096382172), ('D:\\picture\\Training\\IMG_6979.jpg', 0.32341043006191067), ('D:\\picture\\Training\\2023_03_15_17_42_IMG_1774.JPG', 0.003924756366049111), ('D:\\picture\\Training\\2023_03_15_17_42_IMG_1837.JPG', 0.0030302620355680296), ('D:\\picture\\Training\\2023_03_15_17_41_IMG_1772.JPG', 0.0), ('D:\\picture\\Training\\car.jpg', 0.0)]


In [19]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
query_image = "D:\picture\Training\query.jpg"
def display_image_with_boxes(image_path, objects):
    image = Image.open(image_path)
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)
    
    # 绘制检测框
    for obj in objects:
        bbox = obj['bbox']
        rect = patches.Rectangle(
            (bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
            linewidth=2, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(bbox[0], bbox[1] - 10, f"{obj['label']} ({obj['confidence']:.2f})", 
                color='red', fontsize=12, backgroundcolor="white")
    plt.show()
print("Query Image and Detected Objects:")
display_image_with_boxes(query_image, query_objects)
print("\nTop Matches:")
for match in matches[:3]:  # 只展示前 3 个匹配结果
    image_path, similarity = match
    print(f"Matched Image: {image_path}, Similarity: {similarity:.2f}")
    display_image_with_boxes(image_path, database[image_path])

Query Image and Detected Objects:

Top Matches:
Matched Image: D:\picture\Training\query.jpg, Similarity: 3.00
Matched Image: D:\picture\Training\IMG_6996.jpg, Similarity: 0.78
Matched Image: D:\picture\Training\IMG_7018.jpg, Similarity: 0.77


## 使用标注好的数据集来自己训练（可实现性不大）
适用场景：
如果数据集中有公开的高质量分类标签，可以直接使用。
常用数据集：
ImageNet：分类标签数据集，覆盖 1000 多种类别。
COCO Dataset：含目标检测和分类的多标签数据。
Open Images Dataset：包含大量多标签标注图像，支持目标检测

# 利用图像特征提取的方法直接实现

## 利用欧几里得距离（基本原理）

In [ ]:
import cv2
import numpy as np
import os
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt


# 提取颜色直方图特征
def extract_features(image, bins=(8, 8, 8)):
    # 将图像转换为 HSV 颜色空间
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # 计算颜色直方图
    hist = cv2.calcHist([hsv_image], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    # 归一化
    hist = cv2.normalize(hist, hist).flatten()
    return hist


# 索引图像库
def index_images(image_dir, bins=(8, 8, 8)):
    index = {}
    for image_name in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        if image is not None:
            features = extract_features(image, bins)
            index[image_name] = features
    return index


# 搜索与查询图像最相似的图像
def search(query_features, index, top_k=10):
    results = {}
    for image_name, features in index.items():
        distance = euclidean_distances([query_features], [features])[0][0]
        results[image_name] = distance
    # 按距离排序（越小越相似）
    results = sorted(results.items(), key=lambda x: x[1])
    return results[:top_k]


# 显示结果
def show_results(query_image_path, results, image_dir):
    query_image = cv2.imread(query_image_path)
    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(15, 5))
    plt.subplot(1, len(results) + 1, 1)
    plt.imshow(query_image)
    plt.title("Query Image")
    plt.axis("off")
    
    for i, (image_name, score) in enumerate(results, start=2):
        result_image = cv2.imread(os.path.join(image_dir, image_name))
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        plt.subplot(1, len(results) + 1, i)
        plt.imshow(result_image)
        plt.title(f"Rank {i-1}\nScore: {score:.2f}")
        plt.axis("off")
    plt.show()


# 主程序
if __name__ == "__main__":
    # 图像库目录
    image_dir = "D:\picture\Training"
    # 查询图像路径
    query_image_path = "D:\picture\Training\query.jpg"
    
    # 索引图像库
    print("Indexing images...")
    index = index_images(image_dir)
    
    # 读取查询图像并提取特征
    query_image = cv2.imread(query_image_path)
    query_features = extract_features(query_image)
    
    # 搜索最相似的图像
    print("Searching for similar images...")
    results = search(query_features, index, top_k=20)
    
    # 显示结果
    print("Displaying results...")
    show_results(query_image_path, results, image_dir)


## 使用预训练模型
直接使用现成的深度学习模型（如 CNN）提取图像特征向量。\
适用模型：\
ResNet、VGG、EfficientNet：适合通用图像特征提取。\
CLIP (OpenAI)：结合文本和图像，适合更智能的匹配。\
百度飞桨 (PaddlePaddle)：中文支持更好，提供便捷的预训练模型调用。\
特征提取步骤：\
将输入图像传入模型，去掉最后的全连接层，只保留特征层输出。\
得到的特征向量是一个高维数组（如 512 或 2048 维），表示图像的特征信息。\

数据库的建立与管理
存储图像特征：

对图像库中的每张图片提取特征向量，存储到数据库中（如 NumPy 数组文件、SQLite、MongoDB）。
同时保存图片的路径或文件名，便于后续检索时返回结果。
高效检索工具：

Annoy (Approximate Nearest Neighbors)：适合快速近似相似度搜索。
FAISS (Facebook AI Similarity Search)：支持 GPU 加速的大规模特征检索库。
3. 图像检索流程
(1) 特征提取
对用户上传的图像提取特征向量。
(2) 相似度计算
使用相似度度量方法比较查询图片的特征向量和数据库中的特征：
欧几里得距离：适合衡量全局特征差异。
余弦相似度：适合高维特征的匹配（值越接近 1，图像越相似）。
(3) 结果排序
根据相似度分数，对检索结果进行排序，返回最相似的几张图片。

### 特征提取与特征存储

利用CNN从图像中提取特征。可以采用以下两种方法：

预训练模型（Transfer Learning）：
使用现成的预训练模型（如 ResNet、VGG、Inception 等）作为特征提取器。
去掉最后一层分类器，提取中间层或倒数第二层的特征向量。\
自定义模型：
构建自己的 CNN 模型，根据数据集训练，然后从中提取特征。\
示例代码（使用预训练 ResNet 提取特征）：

#### 对图片库进行特征提取

In [ ]:
import os
from torchvision import models, transforms
from PIL import Image
import torch
import numpy as np

# 加载预训练模型用于特征提取
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # 移除分类层
model.eval()

# 图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 定义比对库路径
image_dir = "D:\picture\Training"
features_list = []  # 用于存储特征向量
image_paths = []  # 用于存储图像路径

# 遍历文件夹，提取每张图像的特征
for img_file in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_file)
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)  # 添加 batch 维度
        
    # 核心步骤：利用CNN提取特征
    with torch.no_grad():
        features = model(img_tensor).squeeze().numpy()
        
    features_list.append(features)
    image_paths.append(img_path)

# 将特征保存为 NumPy 数组
features_array = np.array(features_list)
np.save("database_features.npy", features_array)
np.save("database_paths.npy", image_paths)  # 保存图像路径

#### 对查询图像进行提取

In [ ]:
# 查询图像路径
query_image_path = "D:\picture\Training\query.jpg"
# 加载并预处理查询图像
query_img = Image.open(query_image_path).convert("RGB")
query_img_tensor = transform(query_img).unsqueeze(0)
# 提取查询图像的特征
with torch.no_grad():
    query_features = model(query_img_tensor).squeeze().numpy()

### 相似度计算
利用余弦相似度计算

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 加载比对库特征和路径
database_features = np.load("database_features.npy")
database_paths = np.load("database_paths.npy")

# 计算相似度（使用余弦相似度）
similarities = cosine_similarity(query_features.reshape(1, -1), database_features)

# 获取最相似的前 K 张图像
top_k = 5  # 返回前 5 张相似图像
top_k_indices = similarities.argsort()[0][-top_k:][::-1]  # 按相似度降序排列

# 显示结果
for idx in top_k_indices:
    similar_img_path = database_paths[idx]
    print(f"相似图像路径: {similar_img_path}, 相似度: {similarities[0, idx]:.4f}")


### 显示检索结果

In [ ]:
import matplotlib.pyplot as plt

# 显示查询图像
plt.figure(figsize=(10, 5))
plt.subplot(1, top_k + 1, 1)
plt.imshow(Image.open(query_image_path))
plt.title("Query Image")
plt.axis("off")

# 显示相似结果
for i, idx in enumerate(top_k_indices, start=2):
    similar_img_path = database_paths[idx]
    plt.subplot(1, top_k + 1, i)
    plt.imshow(Image.open(similar_img_path))
    plt.title(f"Rank {i-1}")
    plt.axis("off")

plt.show()


# UI封装

## 利用分类标签比对的方法
运行方式
保存代码为 app.py，在终端运行以下命令启动服务：

In [ ]:
import streamlit as st
import io
# Streamlit UI
st.title("基于 YOLO 的图像检索系统")
st.sidebar.header("功能选项")

# 上传查询图像
query_image = st.sidebar.file_uploader("上传查询图片", type=['jpg', 'png'])

# 设置图像数据库路径
image_folder = st.sidebar.text_input("数据库图片文件夹路径", "path_to_your_image_folder")

# 按钮：构建数据库
if st.sidebar.button("构建数据库"):
    if image_folder:
        st.sidebar.write("正在构建数据库，请稍候...")
        database = build_database(image_folder)
        with open("database.json", "w") as f:
            json.dump(database, f)
        st.sidebar.success("数据库已构建完成！")
    else:
        st.sidebar.error("请指定有效的图片文件夹路径。")

# 加载数据库
if st.sidebar.button("加载数据库"):
    try:
        with open("database.json", "r") as f:
            database = json.load(f)
        st.sidebar.success("数据库加载成功！")
    except FileNotFoundError:
        st.sidebar.error("未找到数据库文件，请先构建数据库。")

# 检索并展示结果
if query_image and st.sidebar.button("开始检索"):
    query_image_pil = Image.open(query_image)
    query_image_path = "query_image.jpg"
    query_image_pil.save(query_image_path)

    # 检测查询图像目标
    query_objects = detect_objects(query_image_path)
    st.subheader("查询图像及检测结果")
    st.image(query_image_pil, caption="查询图像", use_column_width=True)

    # 检索相似图像
    matches = match_images(query_objects, database)

    # 显示检索结果
    st.subheader("检索结果")
    for match in matches[:5]:  # 显示前 5 个匹配结果
        matched_image_path, similarity = match
        st.write(f"匹配图像: {matched_image_path}, 相似度: {similarity:.2f}")
        buf = display_image_with_boxes(matched_image_path, database[matched_image_path])
        st.image(buf, caption=f"匹配结果: {matched_image_path}", use_column_width=True)

## 利用CNN提取图像特征的方法

In [ ]:
import os
import numpy as np
import streamlit as st
from PIL import Image
from torchvision import models, transforms
from sklearn.metrics.pairwise import cosine_similarity
import torch
import matplotlib.pyplot as plt

# Streamlit 标题
st.title("基于 CNN 的内容检索图像 (CBIR) 系统")

# 图像预处理和模型加载
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # 移除分类层
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 特征提取函数
def extract_features(img_path_or_object):
    if isinstance(img_path_or_object, str):
        img = Image.open(img_path_or_object).convert("RGB")
    else:  # 如果是 Streamlit 上传的图像文件
        img = Image.open(img_path_or_object).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        return model(img_tensor).squeeze().numpy()

# 准备比对库特征
@st.cache  # 缓存计算结果，提高速度
def prepare_database(image_dir):
    features, paths = [], []
    for img_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_file)
        features.append(extract_features(img_path))
        paths.append(img_path)
    return np.array(features), np.array(paths)

# 显示查询结果
def display_results(query_path_or_object, top_k=5):
    db_features, db_paths = prepare_database(image_dir)
    query_features = extract_features(query_path_or_object)
    
    # 计算相似度
    similarities = cosine_similarity(query_features.reshape(1, -1), db_features)
    top_indices = similarities.argsort()[0][-top_k:][::-1]

    # 显示查询图像
    st.subheader("查询图像")
    if isinstance(query_path_or_object, str):
        st.image(query_path_or_object, caption="查询图像", use_column_width=True)
    else:
        st.image(query_path_or_object, caption="查询图像", use_column_width=True)
    
    # 显示相似图像
    st.subheader("最相似的图像")
    for rank, idx in enumerate(top_indices, start=1):
        similar_img_path = db_paths[idx]
        similarity_score = similarities[0, idx]
        st.image(similar_img_path, caption=f"Rank {rank} - 相似度: {similarity_score:.4f}", use_column_width=True)

# 比对库路径
image_dir = "image_database"

# Streamlit 主界面
st.sidebar.header("操作选项")
uploaded_file = st.sidebar.file_uploader("上传查询图像", type=["jpg", "jpeg", "png"])

if uploaded_file:
    st.sidebar.success("图像上传成功！")
    if st.sidebar.button("开始检索"):
        display_results(uploaded_file)
else:
    st.info("请在左侧上传一张查询图像。")


## 两者结合起来的最终封装